# Implement a simple neural network

In [1]:
import numpy as np

training_data = np.array([[0, 0, 1],
                         [0, 1, 1],
                         [1, 0, 1],
                         [1, 1, 1]])
training_labels = np.array([[0, 1, 1, 0]]).T

In [2]:
'''
This Network class is adapted from Michael Nielsen's code.

A module to implement the minibatch stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
'''
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.initialize_weights()

    def initialize_weights(self):
        self.biases = [np.random.randn(1, y) for y in self.sizes[1:]]
        self.weights = [np.random.randn(x, y)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def SGD(self, training_data, training_labels, epochs, mini_batch_size, eta,
            test_data=None, test_labels=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            unison_shuffle(training_data, training_labels)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            mini_batches_labels = [
                training_labels[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch, mini_batch_labels in zip(mini_batches, mini_batches_labels):
                self.update_mini_batch(mini_batch, mini_batch_labels, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, mini_batch_labels, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        
        y = self.forward(mini_batch)
        dw, db = self.backward(mini_batch, y, mini_batch_labels)
        for i in range(0, self.num_layers - 1):
            self.weights[i] -= eta * dw[i]
            self.biases[i] -= eta * db[i]
        
    def forward(self, X):
        """Return the output of the network if ``a`` is input."""
        self.a = []
        self.a.append(sigmoid(np.dot(X, self.weights[0]) + self.biases[0]))
        for i in range(1, self.num_layers-1):
            self.a.append(sigmoid(self.a[i-1].dot(self.weights[i]) + self.biases[i]))
        return self.a[-1]
    
    def backward(self, X, y, labels):
        """Return a tuple ``(db, dw)`` representing the
        gradient for the cost function C_x.  ``db`` and
        ``dw`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        size = len(X)
        da = [np.zeros(a.shape) for a in self.a]
        dz = [np.zeros(a.shape) for a in self.a]
        db = [np.zeros(b.shape) for b in self.biases]
        dw = [np.zeros(w.shape) for w in self.weights]
        da[-1] = self.cost_derivative(y, labels)
        dz[-1] = sigmoid(self.a[-1], True) * da[-1]
        db[-1] = np.dot(np.ones((1, size)), dz[-1])/size
        for i in range(self.num_layers - 3, -1, -1):
            da[i] = np.dot(dz[i+1], self.weights[i+1].T)
            dz[i] = sigmoid(self.a[i], True) * da[i]
            db[i] = np.dot(np.ones((1, size), dtype=np.int), dz[i])/size
            dw[i+1] = np.dot(self.a[i].T, dz[i+1])/size
        
        dw[0] = np.dot(np.array(X).T, dz[0])/size
        return (dw, db)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        #showImage(test_data[0], test_labels[0])
        #print np.argmax(self.forward(test_data[0]))
        test_results = [(np.argmax(self.forward(x)), y)
                        for (x, y) in zip(test_data, test_labels)]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, outputs, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (outputs-y)

#### Miscellaneous functions
def sigmoid(z, deriv=False):
    """The sigmoid function."""
    if deriv:
        return z*(1-z)
    return 1.0/(1.0+np.exp(-z))

def unison_shuffle(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

Create the neural net

In [3]:
net = Network([3, 4, 1])
net.SGD(training_data, training_labels, 10000, 4, 3.0)

Epoch 0 complete
Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
Epoch 4 complete
Epoch 5 complete
Epoch 6 complete
Epoch 7 complete
Epoch 8 complete
Epoch 9 complete
Epoch 10 complete
Epoch 11 complete
Epoch 12 complete
Epoch 13 complete
Epoch 14 complete
Epoch 15 complete
Epoch 16 complete
Epoch 17 complete
Epoch 18 complete
Epoch 19 complete
Epoch 20 complete
Epoch 21 complete
Epoch 22 complete
Epoch 23 complete
Epoch 24 complete
Epoch 25 complete
Epoch 26 complete
Epoch 27 complete
Epoch 28 complete
Epoch 29 complete
Epoch 30 complete
Epoch 31 complete
Epoch 32 complete
Epoch 33 complete
Epoch 34 complete
Epoch 35 complete
Epoch 36 complete
Epoch 37 complete
Epoch 38 complete
Epoch 39 complete
Epoch 40 complete
Epoch 41 complete
Epoch 42 complete
Epoch 43 complete
Epoch 44 complete
Epoch 45 complete
Epoch 46 complete
Epoch 47 complete
Epoch 48 complete
Epoch 49 complete
Epoch 50 complete
Epoch 51 complete
Epoch 52 complete
Epoch 53 complete
Epoch 54 complete
Epoch 55 complete
Ep

Epoch 477 complete
Epoch 478 complete
Epoch 479 complete
Epoch 480 complete
Epoch 481 complete
Epoch 482 complete
Epoch 483 complete
Epoch 484 complete
Epoch 485 complete
Epoch 486 complete
Epoch 487 complete
Epoch 488 complete
Epoch 489 complete
Epoch 490 complete
Epoch 491 complete
Epoch 492 complete
Epoch 493 complete
Epoch 494 complete
Epoch 495 complete
Epoch 496 complete
Epoch 497 complete
Epoch 498 complete
Epoch 499 complete
Epoch 500 complete
Epoch 501 complete
Epoch 502 complete
Epoch 503 complete
Epoch 504 complete
Epoch 505 complete
Epoch 506 complete
Epoch 507 complete
Epoch 508 complete
Epoch 509 complete
Epoch 510 complete
Epoch 511 complete
Epoch 512 complete
Epoch 513 complete
Epoch 514 complete
Epoch 515 complete
Epoch 516 complete
Epoch 517 complete
Epoch 518 complete
Epoch 519 complete
Epoch 520 complete
Epoch 521 complete
Epoch 522 complete
Epoch 523 complete
Epoch 524 complete
Epoch 525 complete
Epoch 526 complete
Epoch 527 complete
Epoch 528 complete
Epoch 529 co

Epoch 979 complete
Epoch 980 complete
Epoch 981 complete
Epoch 982 complete
Epoch 983 complete
Epoch 984 complete
Epoch 985 complete
Epoch 986 complete
Epoch 987 complete
Epoch 988 complete
Epoch 989 complete
Epoch 990 complete
Epoch 991 complete
Epoch 992 complete
Epoch 993 complete
Epoch 994 complete
Epoch 995 complete
Epoch 996 complete
Epoch 997 complete
Epoch 998 complete
Epoch 999 complete
Epoch 1000 complete
Epoch 1001 complete
Epoch 1002 complete
Epoch 1003 complete
Epoch 1004 complete
Epoch 1005 complete
Epoch 1006 complete
Epoch 1007 complete
Epoch 1008 complete
Epoch 1009 complete
Epoch 1010 complete
Epoch 1011 complete
Epoch 1012 complete
Epoch 1013 complete
Epoch 1014 complete
Epoch 1015 complete
Epoch 1016 complete
Epoch 1017 complete
Epoch 1018 complete
Epoch 1019 complete
Epoch 1020 complete
Epoch 1021 complete
Epoch 1022 complete
Epoch 1023 complete
Epoch 1024 complete
Epoch 1025 complete
Epoch 1026 complete
Epoch 1027 complete
Epoch 1028 complete
Epoch 1029 complete
E

Epoch 1496 complete
Epoch 1497 complete
Epoch 1498 complete
Epoch 1499 complete
Epoch 1500 complete
Epoch 1501 complete
Epoch 1502 complete
Epoch 1503 complete
Epoch 1504 complete
Epoch 1505 complete
Epoch 1506 complete
Epoch 1507 complete
Epoch 1508 complete
Epoch 1509 complete
Epoch 1510 complete
Epoch 1511 complete
Epoch 1512 complete
Epoch 1513 complete
Epoch 1514 complete
Epoch 1515 complete
Epoch 1516 complete
Epoch 1517 complete
Epoch 1518 complete
Epoch 1519 complete
Epoch 1520 complete
Epoch 1521 complete
Epoch 1522 complete
Epoch 1523 complete
Epoch 1524 complete
Epoch 1525 complete
Epoch 1526 complete
Epoch 1527 complete
Epoch 1528 complete
Epoch 1529 complete
Epoch 1530 complete
Epoch 1531 complete
Epoch 1532 complete
Epoch 1533 complete
Epoch 1534 complete
Epoch 1535 complete
Epoch 1536 complete
Epoch 1537 complete
Epoch 1538 complete
Epoch 1539 complete
Epoch 1540 complete
Epoch 1541 complete
Epoch 1542 complete
Epoch 1543 complete
Epoch 1544 complete
Epoch 1545 complete


Epoch 2005 complete
Epoch 2006 complete
Epoch 2007 complete
Epoch 2008 complete
Epoch 2009 complete
Epoch 2010 complete
Epoch 2011 complete
Epoch 2012 complete
Epoch 2013 complete
Epoch 2014 complete
Epoch 2015 complete
Epoch 2016 complete
Epoch 2017 complete
Epoch 2018 complete
Epoch 2019 complete
Epoch 2020 complete
Epoch 2021 complete
Epoch 2022 complete
Epoch 2023 complete
Epoch 2024 complete
Epoch 2025 complete
Epoch 2026 complete
Epoch 2027 complete
Epoch 2028 complete
Epoch 2029 complete
Epoch 2030 complete
Epoch 2031 complete
Epoch 2032 complete
Epoch 2033 complete
Epoch 2034 complete
Epoch 2035 complete
Epoch 2036 complete
Epoch 2037 complete
Epoch 2038 complete
Epoch 2039 complete
Epoch 2040 complete
Epoch 2041 complete
Epoch 2042 complete
Epoch 2043 complete
Epoch 2044 complete
Epoch 2045 complete
Epoch 2046 complete
Epoch 2047 complete
Epoch 2048 complete
Epoch 2049 complete
Epoch 2050 complete
Epoch 2051 complete
Epoch 2052 complete
Epoch 2053 complete
Epoch 2054 complete


...

Epoch 7379 complete
Epoch 7380 complete
Epoch 7381 complete
Epoch 7382 complete
Epoch 7383 complete
Epoch 7384 complete
Epoch 7385 complete
Epoch 7386 complete
Epoch 7387 complete
Epoch 7388 complete
Epoch 7389 complete
Epoch 7390 complete
Epoch 7391 complete
Epoch 7392 complete
Epoch 7393 complete
Epoch 7394 complete
Epoch 7395 complete
Epoch 7396 complete
Epoch 7397 complete
Epoch 7398 complete
Epoch 7399 complete
Epoch 7400 complete
Epoch 7401 complete
Epoch 7402 complete
Epoch 7403 complete
Epoch 7404 complete
Epoch 7405 complete
Epoch 7406 complete
Epoch 7407 complete
Epoch 7408 complete
Epoch 7409 complete
Epoch 7410 complete
Epoch 7411 complete
Epoch 7412 complete
Epoch 7413 complete
Epoch 7414 complete
Epoch 7415 complete
Epoch 7416 complete
Epoch 7417 complete
Epoch 7418 complete
Epoch 7419 complete
Epoch 7420 complete
Epoch 7421 complete
Epoch 7422 complete
Epoch 7423 complete
Epoch 7424 complete
Epoch 7425 complete
Epoch 7426 complete
Epoch 7427 complete
Epoch 7428 complete


Epoch 8058 complete
Epoch 8059 complete
Epoch 8060 complete
Epoch 8061 complete
Epoch 8062 complete
Epoch 8063 complete
Epoch 8064 complete
Epoch 8065 complete
Epoch 8066 complete
Epoch 8067 complete
Epoch 8068 complete
Epoch 8069 complete
Epoch 8070 complete
Epoch 8071 complete
Epoch 8072 complete
Epoch 8073 complete
Epoch 8074 complete
Epoch 8075 complete
Epoch 8076 complete
Epoch 8077 complete
Epoch 8078 complete
Epoch 8079 complete
Epoch 8080 complete
Epoch 8081 complete
Epoch 8082 complete
Epoch 8083 complete
Epoch 8084 complete
Epoch 8085 complete
Epoch 8086 complete
Epoch 8087 complete
Epoch 8088 complete
Epoch 8089 complete
Epoch 8090 complete
Epoch 8091 complete
Epoch 8092 complete
Epoch 8093 complete
Epoch 8094 complete
Epoch 8095 complete
Epoch 8096 complete
Epoch 8097 complete
Epoch 8098 complete
Epoch 8099 complete
Epoch 8100 complete
Epoch 8101 complete
Epoch 8102 complete
Epoch 8103 complete
Epoch 8104 complete
Epoch 8105 complete
Epoch 8106 complete
Epoch 8107 complete


Epoch 8603 complete
Epoch 8604 complete
Epoch 8605 complete
Epoch 8606 complete
Epoch 8607 complete
Epoch 8608 complete
Epoch 8609 complete
Epoch 8610 complete
Epoch 8611 complete
Epoch 8612 complete
Epoch 8613 complete
Epoch 8614 complete
Epoch 8615 complete
Epoch 8616 complete
Epoch 8617 complete
Epoch 8618 complete
Epoch 8619 complete
Epoch 8620 complete
Epoch 8621 complete
Epoch 8622 complete
Epoch 8623 complete
Epoch 8624 complete
Epoch 8625 complete
Epoch 8626 complete
Epoch 8627 complete
Epoch 8628 complete
Epoch 8629 complete
Epoch 8630 complete
Epoch 8631 complete
Epoch 8632 complete
Epoch 8633 complete
Epoch 8634 complete
Epoch 8635 complete
Epoch 8636 complete
Epoch 8637 complete
Epoch 8638 complete
Epoch 8639 complete
Epoch 8640 complete
Epoch 8641 complete
Epoch 8642 complete
Epoch 8643 complete
Epoch 8644 complete
Epoch 8645 complete
Epoch 8646 complete
Epoch 8647 complete
Epoch 8648 complete
Epoch 8649 complete
Epoch 8650 complete
Epoch 8651 complete
Epoch 8652 complete


Epoch 9102 complete
Epoch 9103 complete
Epoch 9104 complete
Epoch 9105 complete
Epoch 9106 complete
Epoch 9107 complete
Epoch 9108 complete
Epoch 9109 complete
Epoch 9110 complete
Epoch 9111 complete
Epoch 9112 complete
Epoch 9113 complete
Epoch 9114 complete
Epoch 9115 complete
Epoch 9116 complete
Epoch 9117 complete
Epoch 9118 complete
Epoch 9119 complete
Epoch 9120 complete
Epoch 9121 complete
Epoch 9122 complete
Epoch 9123 complete
Epoch 9124 complete
Epoch 9125 complete
Epoch 9126 complete
Epoch 9127 complete
Epoch 9128 complete
Epoch 9129 complete
Epoch 9130 complete
Epoch 9131 complete
Epoch 9132 complete
Epoch 9133 complete
Epoch 9134 complete
Epoch 9135 complete
Epoch 9136 complete
Epoch 9137 complete
Epoch 9138 complete
Epoch 9139 complete
Epoch 9140 complete
Epoch 9141 complete
Epoch 9142 complete
Epoch 9143 complete
Epoch 9144 complete
Epoch 9145 complete
Epoch 9146 complete
Epoch 9147 complete
Epoch 9148 complete
Epoch 9149 complete
Epoch 9150 complete
Epoch 9151 complete


Epoch 9613 complete
Epoch 9614 complete
Epoch 9615 complete
Epoch 9616 complete
Epoch 9617 complete
Epoch 9618 complete
Epoch 9619 complete
Epoch 9620 complete
Epoch 9621 complete
Epoch 9622 complete
Epoch 9623 complete
Epoch 9624 complete
Epoch 9625 complete
Epoch 9626 complete
Epoch 9627 complete
Epoch 9628 complete
Epoch 9629 complete
Epoch 9630 complete
Epoch 9631 complete
Epoch 9632 complete
Epoch 9633 complete
Epoch 9634 complete
Epoch 9635 complete
Epoch 9636 complete
Epoch 9637 complete
Epoch 9638 complete
Epoch 9639 complete
Epoch 9640 complete
Epoch 9641 complete
Epoch 9642 complete
Epoch 9643 complete
Epoch 9644 complete
Epoch 9645 complete
Epoch 9646 complete
Epoch 9647 complete
Epoch 9648 complete
Epoch 9649 complete
Epoch 9650 complete
Epoch 9651 complete
Epoch 9652 complete
Epoch 9653 complete
Epoch 9654 complete
Epoch 9655 complete
Epoch 9656 complete
Epoch 9657 complete
Epoch 9658 complete
Epoch 9659 complete
Epoch 9660 complete
Epoch 9661 complete
Epoch 9662 complete


In [4]:
training_data

array([[1, 0, 1],
       [0, 1, 1],
       [1, 1, 1],
       [0, 0, 1]])

In [6]:
net.forward([[1,0,0]])

array([[ 0.99413289]])